In [1]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext, sql
spark = SparkSession.builder.appName('Review Cleaning').getOrCreate()

In [1]:
path = '/Users/wangyinuo/Desktop/DSCI551/project/yelp_dataset/yelp_academic_dataset_review.json'
review_df = spark.read.json(path)
review_df.show(10)

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|-MhfebM0QIsKt87iD...|   0|2015-04-15 05:21:16|    0|xQY8N_XvtGbearJ5X...|  2.0|As someone who ha...|     5|OwjRMXRC0KyPrIlcj...|
|lbrU8StCq3yDfr-QM...|   0|2013-12-07 03:16:52|    1|UmFMZ8PyXZTY2Qcwz...|  1.0|I am actually hor...|     1|nIJD_7ZXHq-FX8byP...|
|HQl28KMwrEKHqhFrr...|   0|2015-12-05 03:18:11|    0|LG2ZaYiOgpr2DK_90...|  5.0|I love Deagan's. ...|     1|V34qejxNsCbcgD8C0...|
|5JxlZaqCnk1MnbgRi...|   0|2011-05-27 05:30:52|    0|i6g_oA9Yf9Y31qt0w...|  1.0|Dismal, lukewarm,...|     0|ofKDkJKXSKZXu5xJN...|
|IS4cv902ykd8wj1TR...|   0|2017-01-14 21:56:57|    0|6TdNDKywdbjoTkize...|  4.0|Oh happy d

In [2]:
from pyspark.sql.functions import desc
import pyspark.sql.functions as fc 

from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

window = Window.partitionBy(review_df['business_id']).orderBy(review_df['useful'].desc())

review_df = review_df.select('*', rank().over(window).alias('rank')).filter(col('rank') <= 1)

In [3]:
review_df.count()

309006

In [4]:
review_df.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+----+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|rank|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+----+
|--9e1ONYQuAa-CB_R...|  74|2019-04-01 02:32:48|   42|60Y3zgfqkgRcpor47...|  4.0|Wanting a good st...|    80|VmQdv7xMWiyejzxJz...|   1|
|-6c_bJblLXUwoWfmY...|   1|2017-10-27 23:17:40|    0|i3d2wy9jW7lca2Vg4...|  1.0|I would not recom...|    32|PK3EmRHbtPxxuJuEy...|   1|
|-ElDqujEn1u64ynQr...|   0|2014-07-07 19:28:06|    0|2ApaSUqsJ5_6-hxY8...|  5.0|Started my custod...|     4|miVLt4_ELUb95v7gZ...|   1|
|-Gh9a15ijNii-8rnp...|   0|2013-12-16 21:37:43|    2|DZxjGpaFaf7OBS_w8...|  5.0|I purchased a Vol...|     4|9xJnNdFzRP6ZophVy...|   1|
|-I06hkMFrX0KBqu61...|   1|2013-01-16 22:25:45|    0|jL